In [3]:
from azure.core.exceptions import ResourceExistsError
from concurrent.futures import ThreadPoolExecutor, as_completed
import os
import pandas as pd
import traceback
import yaml

import sys
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).parent) + '\src')
from adt_sdk import ADTInstance
from utils.utils_adt import get_schema_into_dfs, transform_to_json, create_patch_update


In [4]:
%reload_ext autoreload
%autoreload 2

In [5]:
pd.set_option('display.max_colwidth', 5000)

# Introduction

This notebook runs the script to use synthetic data to:
- create ADT models, twins and relationships.
- update the twins for telemetry stream times-series data from the synthetic data.
- run sample example queries.


# Credential and initialize DT Client

This section serves to create a digital twin connection to an already provisioned ADT instance, denoted by its URL:

In [6]:
# Make sure the instance is set up supporting the SourceTimeStamp feature
ADT_URL = "yizhu2-dhsts-ppe.api.neu.pp.azuredigitaltwins-ppe.net"

The class `ADTInstance` provides a list of the available functions related to Azure Digital Twins in the Azure Python SDK. See https://pypi.org/project/azure-digitaltwins-core/ for more details. 

The initialization of this class runs authentication through CLI credential and creates a new ditigal twins client.

In [ ]:
ADTInstance1 = ADTInstance(ADT_URL, logging_enable=False)

Note: for illustration, we keep `logging_enable=True` (which outputs the ADT API request info) only in the "Create ADT models" section. 

# Load synthetic data

Synthetic data consists of synthetic telemetry time-series signals (current in Amperes) from an imaginary IoT network of connected machines. Concretely, we load the following data:
- ADT models' json files, where the model jsons are in the appropriate schema for ADT, according to DTDL. See the documentation for DTDL here: https://github.com/Azure/opendigitaltwins-dtdl/blob/master/DTDL/v2/dtdlv2.md
- A topology file, which contains the relationships of which machine is connected to which. It contains `relationshipName`, `sourceId`, `targetId` as columns.
- A file for twins' namings and initial properties (keys and corresponding values). It contains `Id`, `ModelId`, `Timestamp`, `Value` as columns.
- A file for the IoT telemetry signals to be used for the twin property updates. It contains `Id`, `Key`, `Timestamp`, `Value` as columns.


In [8]:
models_folder = "../data/models_json"
topology_file = "../data/synthetic_data/topology_continuous_010121_020121.csv"
inittwin_file = "../data/synthetic_data/initial_twins.csv"
updatetwin_file = "../data/synthetic_data/update_stream_continuous_010121_020121.csv"

In [9]:
df_topology = pd.read_csv(topology_file)
df_inittwins = pd.read_csv(inittwin_file)
df_updatetwins = pd.read_csv(updatetwin_file)

# Create ADT models

First, we can check the list of existing ADT models already in out ADT instance.

In [ ]:
# Check models already in ADT instance
ADTInstance1.list_models()

Then, if the some of the models you want to add, already exists, make sure to delete the ADT model(s) before adding the models, using the model json files in the `models_folder`. 

In [ ]:
#Optional: ensure ADT models don't exist beofre creating, else error
# model_id="dtmi:syntheticfactory:machine;1" # 'dtmi:syntheticfactory:machinesimple;1'
# ADTInstance1.delete_model(model_id)

In [11]:
# Create models available in models_folder
list_models=[]
for i in os.listdir(models_folder):
    model_json = yaml.safe_load(open(os.path.join(models_folder,i)))
    list_models.append(model_json)
try:
    ADTInstance1.create_models(list_models)
except ResourceExistsError as e:
    print('ModelId might already exist, Error message: {}'.format(e.error.message))
except Exception as e:
    raise e

ModelId might already exist, Error message: Some of the model ids already exist: dtmi:syntheticfactory:feedmachine2;1, dtmi:syntheticfactory:sourcemachine2;1. Use Model_List API to view models that already exist. See the Swagger example (https://aka.ms/ModelListSwSmpl).


To verify that our models were indeed added to our ADT instance, we run:

In [14]:
# Check models now in ADT instance
ADTInstance1.list_models()


 model  0
{'additional_properties': {}, 'display_name': {'en': 'feedmachine'}, 'description': {'en': 'a smaller source node, i.e. smaller machine, which is connected with machines upstream (and potentially downstream) in our synthetic factory'}, 'id': 'dtmi:syntheticfactory:feedmachine2;1', 'upload_time': datetime.datetime(2022, 2, 2, 17, 13, 14, 437191, tzinfo=<FixedOffset '+00:00'>), 'decommissioned': False, 'model': None}

 model  1
{'additional_properties': {}, 'display_name': {'en': 'sourcemachine'}, 'description': {'en': 'a big source node, i.e. big machine, which receives the supply in our synthetic factory, where machines are linked in the same electric circuit'}, 'id': 'dtmi:syntheticfactory:sourcemachine2;1', 'upload_time': datetime.datetime(2022, 2, 2, 17, 13, 14, 437271, tzinfo=<FixedOffset '+00:00'>), 'decommissioned': False, 'model': None}


We now run the following transformations locally to be able to access the model schema later on.

In [12]:
# Add schema to df_inittwins and df_updatetwins
df_inittwins, df_updatetwins = get_schema_into_dfs(list_models, df_inittwins, df_updatetwins)

In [22]:
df_inittwins.head()

,Id,ModelId,Key,Timestamp,Value,Schema
0,F,dtmi:syntheticfactory:feedmachine2;1,Amps_Ib,2020-12-31 23:59:59.104357,0.0,double
1,E,dtmi:syntheticfactory:feedmachine2;1,Amps_Ib,2020-12-31 23:59:59.240801,0.0,double
2,F,dtmi:syntheticfactory:feedmachine2;1,Amps_Ia,2020-12-31 23:59:59.260915,0.0,double
3,D,dtmi:syntheticfactory:feedmachine2;1,Amps_Ia,2020-12-31 23:59:59.283257,0.0,double
4,E,dtmi:syntheticfactory:feedmachine2;1,PowerMeter,2020-12-31 23:59:59.307807,0.0,double


In [23]:
df_updatetwins.head()

,Id,ModelId,Key,Timestamp,Value,Schema
0,A,dtmi:syntheticfactory:sourcemachine;1,Status,2020-12-31 23:59:59.459645,0,integer
1,B,dtmi:syntheticfactory:sourcemachine;1,Status,2020-12-31 23:59:59.810621,0,integer
2,A,dtmi:syntheticfactory:sourcemachine;1,Status,2021-01-01 00:04:59.706976,0,integer
3,B,dtmi:syntheticfactory:sourcemachine;1,Status,2021-01-01 00:04:59.964369,0,integer
4,A,dtmi:syntheticfactory:sourcemachine;1,Status,2021-01-01 00:09:59.219233,0,integer


# Create ADT twins with properties

To create ADT twins, we need to define the json to initialize the twins with or without default properties, as defined in `df_inittwins`, and according to the twins' DTDL model definitions. 

In [ ]:
#Optional: ensure ADT twins don't exist beofre creating, else error
#list_twin_ids=[]
# for twin_i in set(list_twin_ids):
#     ADTInstance1.delete_digital_twin(twin_i)

In [26]:
list_init_dicts, list_twin_ids = transform_to_json(df_inittwins)
list_twins = []
for i, twindict_i in enumerate(list_init_dicts):
    created_twin = ADTInstance1.upsert_digital_twin(list_twin_ids[i], twindict_i)
    list_twins.append(created_twin)


### Created Digital Twin:
{'$dtId': 'A', '$etag': 'W/"5b2d3f46-b561-47bf-90b7-ebc6a508cfdb"', 'PowerMeter': 0.0, 'Status': 0, 'Amps_Ia': 0.0, 'Amps_Ic': 0.02231950853476, 'Amps_Ib': 0.0, 'PowerLevel': 'Low', '$metadata': {'$model': 'dtmi:syntheticfactory:sourcemachine2;1', 'PowerMeter': {'lastUpdateTime': '2022-02-16T19:31:47.6499133Z'}, 'Status': {'lastUpdateTime': '2022-02-16T19:31:47.6499133Z'}, 'Amps_Ia': {'lastUpdateTime': '2022-02-16T19:31:47.6499133Z'}, 'Amps_Ic': {'lastUpdateTime': '2022-02-16T19:31:47.6499133Z'}, 'Amps_Ib': {'lastUpdateTime': '2022-02-16T19:31:47.6499133Z'}, 'PowerLevel': {'lastUpdateTime': '2022-02-16T19:31:47.6499133Z'}}}
### Created Digital Twin:
{'$dtId': 'B', '$etag': 'W/"0148b72f-7d11-435c-8651-7591d01609d0"', 'Status': 0, 'Amps_Ia': 0.0, 'PowerMeter': 0.0, 'Amps_Ib': 0.0, 'Amps_Ic': 0.0, 'PowerLevel': 'High', '$metadata': {'$model': 'dtmi:syntheticfactory:sourcemachine2;1', 'Status': {'lastUpdateTime': '2022-02-16T19:31:48.2593111Z'}, 'Amps_Ia': {'lastU

In [27]:
print(len(list_twins))
list_twins

10


[{'$dtId': 'A',
  '$etag': 'W/"5b2d3f46-b561-47bf-90b7-ebc6a508cfdb"',
  'PowerMeter': 0.0,
  'Status': 0,
  'Amps_Ia': 0.0,
  'Amps_Ic': 0.02231950853476,
  'Amps_Ib': 0.0,
  'PowerLevel': 'Low',
  '$metadata': {'$model': 'dtmi:syntheticfactory:sourcemachine2;1',
   'PowerMeter': {'lastUpdateTime': '2022-02-16T19:31:47.6499133Z'},
   'Status': {'lastUpdateTime': '2022-02-16T19:31:47.6499133Z'},
   'Amps_Ia': {'lastUpdateTime': '2022-02-16T19:31:47.6499133Z'},
   'Amps_Ic': {'lastUpdateTime': '2022-02-16T19:31:47.6499133Z'},
   'Amps_Ib': {'lastUpdateTime': '2022-02-16T19:31:47.6499133Z'},
   'PowerLevel': {'lastUpdateTime': '2022-02-16T19:31:47.6499133Z'}}},
 {'$dtId': 'B',
  '$etag': 'W/"0148b72f-7d11-435c-8651-7591d01609d0"',
  'Status': 0,
  'Amps_Ia': 0.0,
  'PowerMeter': 0.0,
  'Amps_Ib': 0.0,
  'Amps_Ic': 0.0,
  'PowerLevel': 'High',
  '$metadata': {'$model': 'dtmi:syntheticfactory:sourcemachine2;1',
   'Status': {'lastUpdateTime': '2022-02-16T19:31:48.2593111Z'},
   'Amps_Ia': 

Here, we see that 10 digital twins are created, with the twin-id, and properties ('Amps_Ia', 'Amps_Ib', 'Amps_Ic') as defined in the associated model's DTDL model json. Upon creation, the update time is also recorded in the metadata.

# Add relationships to twins

Relationships are added separately to the twins as follows. We make use of the topology file, using `df_topology`, to determine the source and target twins, as well as the name of the relationships.

In [295]:
df_topology.head()

,relationshipName,sourceId,targetId
0,isParent,B,E
1,isParent,B,F
2,isParent,B,G
3,isParent,B,I
4,isParent,D,H


In [224]:
for i in df_topology.index:
        machine_relationship = {
            "$relationshipId": f'{df_topology.loc[i]["sourceId"]}-{df_topology.loc[i]["relationshipName"]}-{df_topology.loc[i]["targetId"]}',
            "$sourceId": df_topology.loc[i]["sourceId"],
            "$relationshipName": df_topology.loc[i]["relationshipName"],
            "$targetId": df_topology.loc[i]["targetId"]
        }

        ADTInstance1.upsert_relationship(machine_relationship)


### Created relationship with idB-isParent-E
### Created relationship with idB-isParent-F
### Created relationship with idB-isParent-G
### Created relationship with idB-isParent-I
### Created relationship with idD-isParent-H
### Created relationship with idH-isParent-J
### Created relationship with idA-isParent-C
### Created relationship with idA-isParent-D
### Created relationship with idA-isParent-E
### Created relationship with idI-isRedundant-F
### Created relationship with idF-isRedundant-I


In [225]:
ADTInstance1.list_relationships('A')

{'$relationshipId': 'A-isParent-C', '$etag': 'W/"a1711568-93fa-46c8-86cd-cb66a9f4a086"', '$sourceId': 'A', '$relationshipName': 'isParent', '$targetId': 'C'}
{'$relationshipId': 'A-isParent-D', '$etag': 'W/"085248fa-7a04-493f-9dfb-f9f0d261903d"', '$sourceId': 'A', '$relationshipName': 'isParent', '$targetId': 'D'}
{'$relationshipId': 'A-isParent-E', '$etag': 'W/"b19519ed-5f8f-4552-9237-6c6fa4717ec9"', '$sourceId': 'A', '$relationshipName': 'isParent', '$targetId': 'E'}


In [226]:
ADTInstance1.list_relationships('I')

{'$relationshipId': 'I-isRedundant-F', '$etag': 'W/"9cdf8a05-543f-4926-97c3-3e877479511c"', '$sourceId': 'I', '$relationshipName': 'isRedundant', '$targetId': 'F'}


# Example queries

Here, we have some illustrative queries that we can use to check if the twins and relationships are defined as expected.

In [229]:
query_expression = f"""
SELECT * FROM digitaltwins t 
where IS_OF_MODEL('dtmi:syntheticfactory:sourcemachine;1') 
"""
query_result = ADTInstance1.query_twins(query_expression)
query_result

### DigitalTwins:
{'$dtId': 'A', '$etag': 'W/"6814f598-e3cf-49d5-abc9-64ec38d79de5"', 'Amps_Ic': {'sourceTimestamp': '2020-12-31T23:59:59.179364Z', 'Amps_Ic': 0.0}, 'Amps_Ib': {'sourceTimestamp': '2020-12-31T23:59:59.470839Z', 'Amps_Ib': 0.0}, 'PowerLevel': {'sourceTimestamp': '2020-12-31T23:59:59.737262Z', 'PowerLevel': 'Low'}, 'Amps_Ia': {'sourceTimestamp': '2021-01-01T00:00:00.990074Z', 'Amps_Ia': 0.0}, '$metadata': {'$model': 'dtmi:syntheticfactory:sourcemachine2;1', 'Amps_Ic': {'lastUpdateTime': '2021-12-10T00:49:01.6099358Z'}, 'Amps_Ib': {'lastUpdateTime': '2021-12-10T00:49:01.6099358Z'}, 'PowerLevel': {'lastUpdateTime': '2021-12-10T00:49:01.6099358Z'}, 'Amps_Ia': {'lastUpdateTime': '2021-12-10T00:49:01.6099358Z'}}}
{'$dtId': 'B', '$etag': 'W/"1c16c790-a274-47da-b7bd-cf8fe3d699d9"', 'Amps_Ib': {'sourceTimestamp': '2020-12-31T23:59:59.147836Z', 'Amps_Ib': 0.0}, 'PowerLevel': {'sourceTimestamp': '2020-12-31T23:59:59.51408Z', 'PowerLevel': 'High'}, 'Amps_Ia': {'sourceTimestamp': '20

<iterator object azure.core.paging.ItemPaged at 0x1eeede2dc88>

In [230]:
query_expression = f"""
SELECT t,ct FROM digitaltwins t 
JOIN ct RELATED t.isParent 
WHERE ct.$dtId = 'C'
"""
query_result = ADTInstance1.query_twins(query_expression)
print(query_result)

### DigitalTwins:
{'t': {'$dtId': 'A', '$etag': 'W/"6814f598-e3cf-49d5-abc9-64ec38d79de5"', 'Amps_Ic': {'sourceTimestamp': '2020-12-31T23:59:59.179364Z', 'Amps_Ic': 0}, 'Amps_Ib': {'sourceTimestamp': '2020-12-31T23:59:59.470839Z', 'Amps_Ib': 0}, 'PowerLevel': {'sourceTimestamp': '2020-12-31T23:59:59.737262Z', 'PowerLevel': 'Low'}, 'Amps_Ia': {'sourceTimestamp': '2021-01-01T00:00:00.990074Z', 'Amps_Ia': 0}, '$metadata': {'$model': 'dtmi:syntheticfactory:sourcemachine2;1', 'Amps_Ic': {'lastUpdateTime': '2021-12-10T00:49:01.6099358Z'}, 'Amps_Ib': {'lastUpdateTime': '2021-12-10T00:49:01.6099358Z'}, 'PowerLevel': {'lastUpdateTime': '2021-12-10T00:49:01.6099358Z'}, 'Amps_Ia': {'lastUpdateTime': '2021-12-10T00:49:01.6099358Z'}}}, 'ct': {'$dtId': 'C', '$etag': 'W/"9af3b6b1-1687-4317-bfe5-82639588d5d3"', 'Amps_Ia': {'sourceTimestamp': '2021-01-01T00:00:00.379707Z', 'Amps_Ia': 0}, 'Amps_Ib': {'sourceTimestamp': '2021-01-01T00:00:00.96584Z', 'Amps_Ib': 0}, 'Amps_Ic': {'sourceTimestamp': '2021-01-

In [231]:
query_expression = f"""
SELECT t,ct FROM digitaltwins t 
JOIN ct RELATED t.isRedundant
WHERE ct.$dtId = 'I'
"""
query_result = ADTInstance1.query_twins(query_expression)
query_result

### DigitalTwins:
{'t': {'$dtId': 'F', '$etag': 'W/"2d68cd19-1ad8-409f-a6ed-4c6ff9b83c2b"', 'Amps_Ia': {'sourceTimestamp': '2021-01-01T00:00:00.158935Z', 'Amps_Ia': 0}, 'Amps_Ib': {'sourceTimestamp': '2021-01-01T00:00:00.247246Z', 'Amps_Ib': 0}, 'Amps_Ic': {'sourceTimestamp': '2021-01-01T00:04:59.336284Z', 'Amps_Ic': 0}, '$metadata': {'$model': 'dtmi:syntheticfactory:feedmachine2;1', 'Amps_Ia': {'lastUpdateTime': '2021-12-10T00:49:02.8916058Z'}, 'Amps_Ib': {'lastUpdateTime': '2021-12-10T00:49:02.8916058Z'}, 'Amps_Ic': {'lastUpdateTime': '2021-12-10T00:49:02.8916058Z'}}}, 'ct': {'$dtId': 'I', '$etag': 'W/"21e127b3-3f86-4ff7-abfb-34a2c1628acf"', 'Amps_Ic': {'sourceTimestamp': '2020-12-31T23:59:59.106646Z', 'Amps_Ic': 0}, 'Amps_Ia': {'sourceTimestamp': '2020-12-31T23:59:59.525303Z', 'Amps_Ia': 0}, 'Amps_Ib': {'sourceTimestamp': '2020-12-31T23:59:59.621948Z', 'Amps_Ib': 0}, '$metadata': {'$model': 'dtmi:syntheticfactory:feedmachine2;1', 'Amps_Ic': {'lastUpdateTime': '2021-12-10T00:49:03.41

<iterator object azure.core.paging.ItemPaged at 0x1eeede23f60>

In [232]:
query_expression = f"""
SELECT t,ct FROM digitaltwins t 
JOIN ct RELATED t.isRedundant
WHERE ct.$dtId = 'F'
"""
query_result = ADTInstance1.query_twins(query_expression)
query_result

### DigitalTwins:
{'t': {'$dtId': 'I', '$etag': 'W/"21e127b3-3f86-4ff7-abfb-34a2c1628acf"', 'Amps_Ic': {'sourceTimestamp': '2020-12-31T23:59:59.106646Z', 'Amps_Ic': 0}, 'Amps_Ia': {'sourceTimestamp': '2020-12-31T23:59:59.525303Z', 'Amps_Ia': 0}, 'Amps_Ib': {'sourceTimestamp': '2020-12-31T23:59:59.621948Z', 'Amps_Ib': 0}, '$metadata': {'$model': 'dtmi:syntheticfactory:feedmachine2;1', 'Amps_Ic': {'lastUpdateTime': '2021-12-10T00:49:03.4109986Z'}, 'Amps_Ia': {'lastUpdateTime': '2021-12-10T00:49:03.4109986Z'}, 'Amps_Ib': {'lastUpdateTime': '2021-12-10T00:49:03.4109986Z'}}}, 'ct': {'$dtId': 'F', '$etag': 'W/"2d68cd19-1ad8-409f-a6ed-4c6ff9b83c2b"', 'Amps_Ia': {'sourceTimestamp': '2021-01-01T00:00:00.158935Z', 'Amps_Ia': 0}, 'Amps_Ib': {'sourceTimestamp': '2021-01-01T00:00:00.247246Z', 'Amps_Ib': 0}, 'Amps_Ic': {'sourceTimestamp': '2021-01-01T00:04:59.336284Z', 'Amps_Ic': 0}, '$metadata': {'$model': 'dtmi:syntheticfactory:feedmachine2;1', 'Amps_Ia': {'lastUpdateTime': '2021-12-10T00:49:02.89

<iterator object azure.core.paging.ItemPaged at 0x1eeede292b0>

# Run updates to twins

To mimic IoT sensor signals flowing through the ADT twins, we update the corresponding twin properties using json patches. We use the time-series in `df_inittwins` to create patches for each data-point. 

In [245]:
df_inittwins.head()

,Id,ModelId,Key,Timestamp,Value,Schema
0,I,dtmi:syntheticfactory:feedmachine2;1,Amps_Ic,2020-12-31 23:59:59.106646,0.0,double
1,B,dtmi:syntheticfactory:sourcemachine2;1,Amps_Ib,2020-12-31 23:59:59.147836,0.0,double
2,A,dtmi:syntheticfactory:sourcemachine2;1,Amps_Ic,2020-12-31 23:59:59.179364,0.0,double
3,E,dtmi:syntheticfactory:feedmachine2;1,Amps_Ib,2020-12-31 23:59:59.216322,0.0,double
4,D,dtmi:syntheticfactory:feedmachine2;1,Amps_Ib,2020-12-31 23:59:59.349022,0.0,double
5,A,dtmi:syntheticfactory:sourcemachine2;1,Amps_Ib,2020-12-31 23:59:59.470839,0.0,double
6,B,dtmi:syntheticfactory:sourcemachine2;1,PowerLevel,2020-12-31 23:59:59.514080,High,string
7,I,dtmi:syntheticfactory:feedmachine2;1,Amps_Ia,2020-12-31 23:59:59.525303,0.0,double
8,B,dtmi:syntheticfactory:sourcemachine2;1,Amps_Ia,2020-12-31 23:59:59.584866,0.0,double
9,I,dtmi:syntheticfactory:feedmachine2;1,Amps_Ib,2020-12-31 23:59:59.621948,0.0,double


We can run each patch update in series:

In [13]:
%%time
update_retries = 5
for i, row in df_updatetwins[:30].iterrows():
        if i%100==0:
            print(f"Running twin update:{i}")
        for try_i in range(update_retries):
            try:
                twin_id, key, patch = create_patch_update(row)
                ADTInstance1.update_digital_twin(twin_id, patch)
            except Exception as e:
                print(f'{type(e).__name__}: {e}')
                traceback.print_exc()
                continue
            else:
                break
        else:
            print(f'All {update_retries} retries failed for update {i}:{row}')

Running twin update:0
CPU times: total: 297 ms
Wall time: 3.97 s


Or using multi-threading:

In [ ]:
%%time
update_list = df_updatetwins[30:].index.tolist() 
update_retries = 5
arg_max_workers = 20

def func_updatepatch(update_i,row):
    if update_i%200==0:
            print(f"Running twin update:{update_i}")
    for try_i in range(update_retries):
        try:
            twin_id, key, patch = create_patch_update(row)
            ADTInstance1.update_digital_twin(twin_id, patch)
        except Exception as e:
            print(f'{type(e).__name__}: {e}')
            traceback.print_exc()
            continue
        else:
            break
    else:
        print(f'All {update_retries} retries failed for update {i}:{row}')

def runner():
    threads= []
    with ThreadPoolExecutor(max_workers=arg_max_workers) as executor:
        for update_i in update_list:
            threads.append(executor.submit(func_updatepatch, update_i, df_updatetwins.loc[update_i,:]))
            
        for task in as_completed(threads):
            if task.result() is not None:
                print(task.result()) 
            
runner()

# Further steps using Kusto/ADX

With the property updates in ADT now done, we can store them in a Kusto table using ADT's Data History feature. The Data History Connection was set up for the current ADT instance, see details about this feature in its [OneNote page](https://microsoft.sharepoint.com/teams/Azure_IoT/_layouts/OneNote.aspx?id=%2Fteams%2FAzure_IoT%2FSmart%20Buildings%2FNotebook%2FSmart%20Buildings&wd=target%28Areas%20v2%2FData%20History.one%7C992D7290-8337-488D-A011-4D6656CD5C74%2F11%5C%2F17%5C%2F21%20Update%7C61336CDD-A2D4-F149-BF31-833A3E7CFF12%2F%29).

Using the Data History table look at it and thereupon run analysis. An example Kusto query is as follows, where `(adt_dh_kaipkiun2DhAdtInstance_westcentralus` is the name of the table created by the Data History connection.

In [ ]:
let ADTendpoint = "https://kaipkiun2DhAdtInstance.api.wcus.digitaltwins.azure.net";
let ADTquery = ```SELECT t.$dtId as tid FROM DIGITALTWINS t where IS_OF_MODEL('dtmi:syntheticfactory:sourcemachine;1')```;
evaluate azure_digital_twins_query_request(ADTendpoint, ADTquery)
| extend Id = tostring(tid)
| join kind=inner (adt_dh_kaipkiun2DhAdtInstance_westcentralus) on Id
| project Id, TimeStamp, Key, Value
| evaluate bag_unpack(Value)
| project sourceTimestamp, Id, Amps_Ia
| where Amps_Ia!=""
| render timechart 


# END